In [270]:
##importing required libraries
import urllib
import oauth2 as oauth
import numpy as np
#from twitter import twitter Api
import json
import tweepy
import pandas as pd
from datetime import datetime, date, time
##sqlalchemy writes records to sqldatabase from pandas dataframe
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

import sqlite3
conn = sqlite3.connect(':memory:')
c = conn.cursor()

In [271]:
##Tweepy authentication keys
consumer_key = 'iWITOrHfpfv68LsvlQizVKFxy'
consumer_secret = 'oMbcTwv5SwwFboe52jYlMMJg2mveWYSNx0n81C3RFv8z8Yn1sY'
access_token = '1094684633244094465-UVMHQ652DNqBsr5TNOLfRKlMZJxV7y'
access_token_secret = 'YI6yaKszS8FXPTZ1tlg79NHNsrm8vZP5y8HqGXjLAlOmK'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret,)

api = tweepy.API(auth)

In [272]:
#Retriving data from user timelines
results = api.user_timeline(screen_name= 'elonmusk', count=60)
results1 = api.user_timeline(screen_name= 'tesla', count=20)
results2 = api.user_timeline(screen_name= 'SpaceX', count=20)
results3 = api.user_timeline(screen_name= 'boringcompany', count=20)
results4 = api.user_timeline(screen_name= 'PayPal', count=20)
# entity= pd.DataFrame(my_demo_list, columns = ['TweetId']) 

In [273]:
def json_file(screen_name,count):
    result = api.user_timeline(screen_name, count)
my_list_of_dicts = []
for each_json_tweet in results:
    my_list_of_dicts.append(each_json_tweet._json)
    
    
 

    
 
  

In [274]:
#-----------------------------------------------------------------------
# Retrieving User's timeline
# 
#-----------------------------------------------------------------------
#Calling a function
json_file('elonmusk', 60)
    
with open('tweet_json.txt', 'w') as file:
        file.write(json.dumps(my_list_of_dicts, indent=4))
my_demo_list= []
with open('tweet_json.txt', encoding='utf-8') as json_file:  
    all_data = json.load(json_file)
    
        
        
    for each_dictionary in all_data:
        UserTweetId = each_dictionary['id']
        Favourites_count = each_dictionary['user']['favourites_count']
        Tweets = each_dictionary['text']
        Favourites_tweets = each_dictionary['favorite_count']
        Followers_count = each_dictionary['user']['followers_count']
        Retweet_count = each_dictionary['retweet_count']
        TweetTime = each_dictionary['created_at']
        UserId = each_dictionary['user']['id']
        Name =each_dictionary['user']['name'].split(' ', 1)
        FirstName = Name[0]
        LastName=Name[1]
        
        #Hashtags = each_dictionary['entities']['hashtags']       
        my_demo_list.append({'UserTweetId': str(UserTweetId),
                             'Tweets': str(Tweets),
                             'Favourites_count': int(Favourites_count),
                             'Favourites_tweets': int(Favourites_tweets),
                             'Retweet_count': int(Retweet_count),
                             'UserId': int(UserId),
                             #'Hashtags' : str(Hashtags),
                             'Followers_count' : int(Followers_count),
                             'FirstName' : str(FirstName),
                             'LastName' : str(LastName),
                             'TweetTime': datetime.strptime(TweetTime,'%a %b %d %H:%M:%S +0000 %Y')
                            })
        
        producer = pd.DataFrame(my_demo_list, columns = ['UserTweetId','Tweets', 'TweetTime' , 'Retweet_count','UserId', 'FirstName', 'LastName','Followers_count',
                                                   'Favourites_tweets', 'Favourites_count'
                                                   #'Hashtags'
                                                    
                                                    ]) 

In [275]:
 producer = pd.DataFrame(my_demo_list, columns = ['UserTweetId','Tweets', 'TweetTime' , 'Retweet_count','UserId', 'FirstName', 'LastName','Followers_count',
                                                   'Favourites_tweets', 'Favourites_count'
                                                   #'Hashtags'
                                                    
                                                    ]) 

#Checking the first Normal Form
#No Repeating groups = Done
#Data Values are atomic = Done
#Each field has different name = Done
#It has a primary key = Done


In [276]:
#-------------------------------------------------
#    Checking the second normal form (2NF)
#
#-------------------------------------------------
producer = pd.DataFrame(my_demo_list, columns = ['UserTweetId','Tweets', 'TweetTime' , 'Retweet_count','UserId', 'FirstName', 'LastName','Followers_count',
                                                   'Favourites_tweets', 'Favourites_count'
                                                 ]) 

#Converting producer into second normal form
Tweets = pd.DataFrame(my_demo_list, columns = ['UserTweetId', 'Tweets', 'TweetTime' , 'Favourites_tweets', 'Retweet_count', 'UserId'  ])



User = pd.DataFrame(my_demo_list, columns = ['UserId', 'FirstName', 'LastName', 'Followers_count', 'Favourites_count' ]) 


#It is in the First Normal Form =Done
#All non key attributes are dependent on ALL parts of primary key =Done
#Each Field has a unique name =Done
#It has a primary key  =Done

In [277]:
#---------------------------------------
#   Checking third normal form
#---------------------------------------

Tweets = pd.DataFrame(my_demo_list, columns = ['UserTweetId', 'Tweets', 'TweetTime' , 'Favourites_tweets', 'Retweet_count'  ])

#Converting Tweets into third normal form
User_tweet_info = pd.DataFrame(my_demo_list, columns = [ 'UserTweetId' ,'Favourites_tweets', 'Retweet_count' ])
Tweets = pd.DataFrame(my_demo_list, columns = ['UserTweetId', 'Tweets', 'TweetTime' , 'UserId'])

#It should be in second normal form = Done
#All non-key attribute are not dependent on other non key attribute = Done
#Each field has a unique name = Done
#Each field has a primary key = Done

In [280]:
#-------------------------------------
# TABLE ON USER
#-------------------------------------
import sqlite3
conn = sqlite3.connect(':memory:')
c = conn.cursor()
User.to_sql('User', con=conn, if_exists='replace')
User_tweet_info.to_sql('User_tweet_info', con=conn, if_exists='replace')
Tweets.to_sql('Tweets', con=conn, if_exists='replace')

In [281]:
#-----------------------------
# VIEW ON USER
#-----------------------------

c.execute("CREATE VIEW IF NOT EXISTS Elon_View AS SELECT tp.Tweets, up.Favourites_tweets, tp.TweetTime FROM User_tweet_info up JOIN Tweets tp  ON up.UserTweetId = tp.UserTweetId")
pd.read_sql_query("SELECT  Tweets, Favourites_tweets, TweetTime  FROM Elon_View limit 3", con=conn)

,Tweets,Favourites_tweets,TweetTime
0,RT @Teslarati: First look at a Tesla Model 3 d...,0,2019-03-23 16:19:24
1,RT @InsideEVs: Tesla Model 3 Easily Outsold Al...,0,2019-03-22 13:43:41
2,@gadgetkc @ideasforEM @Tesla About to be upgra...,1822,2019-03-22 12:58:44


In [282]:
#-----------------------------
# Index ON USER
#-----------------------------

c.execute("CREATE INDEX IF NOT EXISTS IX_Elon_index_UserTweetId  ON User_tweet_info (UserTweetId ASC)")



In [283]:
#-----------------------------
# Function ON USER
#-----------------------------


#c.execute("CREATE function Elon_Funct (i_TweetId number) RETURN varchar2 AS v_Tweets varchar2(50); BEGIN SELECT Tweets INTO v_Tweets FROM Tweets WHERE UserTweetId = i_UserTweetId; RETURN v_Tweets; END;")

In [284]:
#-----------------------------
# Procedure ON USER
#-----------------------------


# c.execute("CREATE PROCEDURE IF NOT EXISTS sp_TWEET_COUNT CURSOR c_tweet_count IS SELECT count(tp.Tweets), up.Favourites_tweets, tp.TweetTime FROM User_tweet_info up JOIN Tweets tp  ON up.UserTweetId = tp.UserTweetId begin FOR r_c_tweet_count IN c_tweet_count LOOP UPDATE Tweets SET Tweets = ' ' ); END LOOP; END;" ) 


In [285]:
#-----------------------------------------------------------------------
# Retrieving user's friends
#-----------------------------------------------------------------------

username = "elonmusk"


query = api.friends(screen_name = username, count = 50)
my_list_of_friends = []
for each_friends in query:
    my_list_of_friends.append(each_friends._json)
    with open('friends_json.txt', 'w') as file:
        file.write(json.dumps(my_list_of_friends, indent=4))
        my_friends_list = []
with open('friends_json.txt', encoding='utf-8') as friends_file:  
    friends_data = json.load(friends_file)
   
    
    for each_friend in friends_data:
     
        User_FriendId = each_friend['id']
        User_FriendsName =each_friend['name']
        Location = each_friend['location'].split('&', 1)
        Location1 = Location[0]
        Location2 = Location1.split(',',1)
        Location3 = Location2[0]
        #Location4 = Location2[1]
        Description = each_friend['description']
        Favourites = each_friend['favourites_count']
        Followers=each_friend['followers_count']
        Friends=each_friend['friends_count']
        CreatedTime = each_friend['created_at']
        StatusId = each_friend['status']['id']
        Status = each_friend['status']['text']
        Status_createdTime = each_friend['status']['created_at']
#         UserId = each_dictionary['user_mentions']['id']
#         Name = each_dictionary['entities']['user_mentions']['name']
        
        
        my_friends_list.append({'User_FriendId': str(User_FriendId),
                             'User_FriendsName': str(User_FriendsName),
                             'Location':    str(Location3),
                             #'Location4' : str(Location4),
                             'Description': str(Description),
                             'Followers': int(Followers),
                             'Favourites': int(Favourites),
                             'Friends': int(Friends),                             
                             'CreatedTime': datetime.strptime(CreatedTime,'%a %b %d %H:%M:%S +0000 %Y'),
                             'StatusId' : int(StatusId),
                             'Status' : str(Status),
                             'Status_createdTime': datetime.strptime(Status_createdTime,'%a %b %d %H:%M:%S +0000 %Y')
                            })
        
        Consumer1 = pd.DataFrame(my_friends_list, columns = 
                                  [ 'User_FriendId','User_FriendsName','Location','Followers', 'Favourites', 'Friends' ,
                                   'Description' ,'CreatedTime', 'StatusId','Status' , 'Status_createdTime'  
                                   
                                  ])

In [286]:
#Checking the first Normal Form

Consumer1 = pd.DataFrame(my_friends_list, columns = 
                                  [ 'User_FriendId','User_FriendsName','Location','Followers', 'Favourites', 'Friends' ,
                                   'Description' ,'CreatedTime', 'StatusId','Status' , 'Status_createdTime'  
                                   
                                  ])

#No Repeating groups = Done
#Data Values are atomic = Done
#Each field has different name = Done
#It has a primary key = Done

In [287]:
#Checking the second normal form (2NF)


Consumer1 = pd.DataFrame(my_friends_list, columns = 
                                  [ 'User_FriendId','User_FriendsName','Location', 'Description','Followers', 'Favourites', 'Friends' 
                                    ,'CreatedTime', 'StatusId','Status' , 'Status_createdTime'  
                                   
                                  ])

#Converting Consumer1 into second form

Friends_db = pd.DataFrame(my_friends_list, columns = 
                                  [ 'User_FriendId','User_FriendsName','Location', 'Description','Followers', 'Favourites', 'Friends' , 'CreatedTime'
                                   ])
Friends_Status = pd.DataFrame(my_friends_list, columns = ['StatusId','Status' , 'Status_createdTime', 'User_FriendId'])



#It is in the First Normal Form =Done
#All non key attributes are dependent on ALL parts of primary key =Done
#Each Field has a unique name = done
#It has a primary key = done


In [288]:
#--------------------------------
#Checking third normal form
#--------------------------------
Friends_db = pd.DataFrame(my_friends_list, columns = 
                                  [ 'User_FriendId','User_FriendsName','Location', 'Description','Followers', 'Favourites', 'Friends', 'CreatedTime' 
                                   ])
#Converting Friends_db into third normal form

Friends_profile = pd.DataFrame(my_friends_list, columns = 
                                  [ 'User_FriendId','User_FriendsName','Location', 'Description' , 'CreatedTime'
                                   ])
Friends_info = pd.DataFrame(my_friends_list, columns = 
                                  [ 'User_FriendId','Followers', 'Favourites', 'Friends'])

#It should be in second normal form = Done
#All non-key attribute are not dependent on other non key attribute =Done
#Each field has a unique name= done
#Each field has a primary key = done

In [291]:
#-------------------------------------
# TABLE ON Friends
#-------------------------------------
import sqlite3
conn = sqlite3.connect(':memory:')
c = conn.cursor()
Friends_profile.to_sql('Friends_profile', con=conn, if_exists='replace')
Friends_info.to_sql('Friends_info', con=conn, if_exists='replace')
Friends_Status.to_sql('Friends_Status', con=conn, if_exists='replace')


In [292]:
#-----------------------------
# VIEW ON Friends
#-----------------------------


c.execute("CREATE VIEW IF NOT EXISTS Elon_Friends AS SELECT fp.User_FriendsName, fp.Location, fp.CreatedTime,  fin.Followers, fin.Friends FROM Friends_profile fp JOIN Friends_info fin ON fp.User_FriendId = fin.User_FriendId")
pd.read_sql_query("SELECT  User_FriendsName, Location, Followers , Friends  FROM Elon_Friends limit 3", con=conn)

,User_FriendsName,Location,Followers,Friends
0,Mars,The Milkyway Galaxy,19448,52
1,The Onion,,11079759,14
2,BBC Science News,London,1377740,79


In [293]:
#-----------------------------
# Index ON Friends
#-----------------------------

c.execute("CREATE INDEX IF NOT EXISTS IX_Elon_Friends_Followers  ON Friends_info  (Followers ASC)")



In [294]:
#-----------------------------
# Function ON Friends
#-----------------------------


#c.execute("CREATE function Elon_Friend_funct (i_User_FriendId number) RETURN varchar2 AS v_Location varchar2(50); BEGIN SELECT Location INTO v_Location FROM Friends_profile WHERE User_FriendId = i_User_FriendId; RETURN v_Location; END;")

In [295]:
#-----------------------------
# Procedure ON Friends
#-----------------------------


# c.execute("CREATE PROCEDURE IF NOT EXISTS sp_Followers_COUNT CURSOR c_follower_count IS SELECT fp.User_FriendsName, fp.Location, fp.CreatedTime,  count(fin.Followers), fin.Friends FROM Friends_profile fp JOIN Friends_info fin ON fp.User_FriendId = fin.User_FriendId begin FOR r_c_follower_count IN c_follower_count LOOP UPDATE Friends_profile SET User_FriendsName = ' ' ); END LOOP; END;" ) 


In [296]:
#-----------------------------------------------------------------------
# Retrieving user's followers
#-----------------------------------------------------------------------
username = "elonmusk"

query = api.followers(screen_name = username, count = 50)
my_list_of_followers = []
for each_followers in query:
    my_list_of_followers.append(each_followers._json)
    with open('followers_json.txt', 'w') as file:
        file.write(json.dumps(my_list_of_followers, indent=4))
        my_followers_list = []
with open('followers_json.txt', encoding='utf-8') as followers_file:  
    followers_data = json.load(followers_file)

    print(followers_data) 
    for each_followers in followers_data:
        user_followersId = each_followers['id']
        Name =each_followers['name'].split(' ')
        user_FollowerFirstName =Name[0]
        #user_FollowerLastName =Name[1]
        user_followerScreenName =each_followers['screen_name']
        Location1 = each_followers['location'].split(',', 1)
        Location=Location1[0]
        Description = each_followers['description']
        Favourites = each_followers['favourites_count']
        Followers=each_followers['followers_count']
        Friends=each_followers['friends_count']
        CreatedTime = each_followers['created_at']
        StatusId = each_followers['statuses_count']
         
    
#         Status = each_followers['status']['text']
#         Status_createdTime = each_followers['status']['created_at']
        
# #         UserId = each_dictionary['user_mentions']['id']
# #         Name = each_dictionary['entities']['user_mentions']['name']
        
        
        my_followers_list.append({'User_FollowersId': str(user_followersId),
                             'User_FollowerFirstName': str(user_FollowerFirstName),
                             #'User_FollowerLastName': str(user_FollowerLastName),
                             'User_FollowerScreenName': str(user_followerScreenName),
                             'Location': str(Location),
                             'Description': str(Description),
                             'Followers': int(Followers),
                             'Favourites': int(Favourites),
                             'Friends': int(Friends),                  
                             'CreatedTime': datetime.strptime(CreatedTime,'%a %b %d %H:%M:%S +0000 %Y'),
                             'StatusCount' : int(StatusId),
#                              'Status' : str(Status),
#                              'Status_createdTime': datetime.strptime(Status_createdTime,'%a %b %d %H:%M:%S +0000 %Y')
                                
                            })
        
        Consumer2 = pd.DataFrame(my_followers_list, columns = 
                                  [ 'User_FollowersId','User_FollowerFirstName',  'User_FollowerScreenName','Location','Followers', 'Favourites','Friends'
                                   ,'Description', 'CreatedTime' 
                               , 'StatusId'
#                                    'Status',   'Status_createdTime'                            
                                   #,'UserId', 'Name'
                                   ])
 

[{'id': 3456810561, 'id_str': '3456810561', 'name': 'King K', 'screen_name': 'KMokgaotsane', 'location': 'outchea on the streets', 'description': 'I keep it Ben Franklin(💯) all day errday.           \n\n\n\n\nInsta:- kgotla_makaveli', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 303, 'friends_count': 485, 'listed_count': 0, 'created_at': 'Thu Aug 27 10:45:47 +0000 2015', 'favourites_count': 814, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 626, 'lang': 'en', 'status': {'created_at': 'Sat Mar 23 22:38:36 +0000 2019', 'id': 1109585262567395328, 'id_str': '1109585262567395328', 'text': 'Next week will be with my niggas, Di nako nyana tse go tla a bo e le gone go ikgogelwang malapeng 😂😂😂', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_re

In [297]:
#--------------------------------
#Checking the first Normal Form
#----------------------------------

Consumer2 = pd.DataFrame(my_followers_list, columns = 
                                  [ 'User_FollowersId','User_FollowerFirstName',  'User_FollowerScreenName','Location','Followers', 'Favourites','Friends'
                                   ,'Description', 'CreatedTime' , 'StatusId', 'Status',   'Status_createdTime'                            
                                   #,'UserId', 'Name'
                                   ])

#No Repeating groups = Done
#Data Values are atomic = Done
#Each field has different name = Done
#It has a primary key = Done

In [298]:
#---------------------------------------
#Checking the second normal form (2NF)
#-----------------------------------------

Consumer2 = pd.DataFrame(my_followers_list, columns = 
                                  [ 'User_FollowersId','User_FollowerFirstName',  'User_FollowerScreenName','Location','Followers', 'Favourites','Friends'
                                   ,'Description', 'CreatedTime' , 'StatusId', 'Status',   'Status_createdTime'                            
                                   #,'UserId', 'Name'
                                   ])

#Converting Consumer2 into second normal form

Followers = pd.DataFrame(my_followers_list, columns = 
                                  [ 'User_FollowersId','User_FollowerFirstName',  'User_FollowerScreenName','Location','Description','Followers', 'Favourites','Friends'
                                   , 'CreatedTime'])
Followers_Tweet = pd.DataFrame(my_followers_list, columns = ['StatusId', 'Status',   'Status_createdTime', 'User_FollowersId'])

#It is in the First Normal Form =Done
#All non key attributes are dependent on ALL parts of primary key =Done
#Each Field has a unique name = done
#It has a primary key = done

In [299]:
#------------------------------
#Checking third normal form
#-------------------------------
Followers = pd.DataFrame(my_followers_list, columns = 
                                  [ 'User_FollowersId','User_FollowerFirstName',  'User_FollowerScreenName','Location','Description','Followers', 'Favourites','Friends'
                                   , 'CreatedTime'])

#Converting Followers  dataframe into three normal form
Followers_profile = pd.DataFrame(my_followers_list, columns = 
                                  [ 'User_FollowersId','User_FollowerFirstName',  'User_FollowerScreenName','Location','Description' , 'CreatedTime'])

Followers_info = pd.DataFrame(my_followers_list, columns = 
                                  [ 'User_FollowersId', 'Followers', 'Favourites','Friends'])


#It should be in second normal form = Done
#All non-key attribute are not dependent on other non key attribute =Done
#Each field has a unique name= done
#Each field has a primary key = done

In [300]:
#-------------------------------------
# TABLE ON Followers
#-------------------------------------

import sqlite3
conn = sqlite3.connect(':memory:')
c = conn.cursor()
Space_profile.to_sql('Space_profile', con=conn, if_exists='replace')
S_tweets.to_sql('S_tweets', con=conn, if_exists='replace')
S_tweets_info.to_sql('S_tweets_info', con=conn, if_exists='replace')


In [301]:
#-----------------------------
# VIEW ON Followers
#-----------------------------

c.execute("CREATE VIEW IF NOT EXISTS Elon_Space AS SELECT ts.Tweets, ts.Space_TweetId,  ins.Retweet_count, ins.Favourite_Tweets_count FROM S_tweets_info ins JOIN S_tweets ts ON ins.Space_TweetId = ts.Space_TweetId")
pd.read_sql_query("SELECT  Space_TweetId, Retweet_count, Favourite_Tweets_count FROM Elon_Space limit 3", con=conn)

DatabaseError: Execution failed on sql 'SELECT  Space_TweetId, Retweet_count, Favourite_Tweets_count FROM Elon_Space limit 3': no such column: ins.Space_TweetId

In [302]:
#-----------------------------------------------------------------------
# Retrieving Search data for User's company space X timeline
# 
#-----------------------------------------------------------------------
my_list_of_dicts = []
for each_json_tweet in results2:
    my_list_of_dicts.append(each_json_tweet._json)
    with open('tweet_json.txt', 'w') as file:
        file.write(json.dumps(my_list_of_dicts, indent=4))
        my_demo_list= []
with open('tweet_json.txt', encoding='utf-8') as json_file: 
    all_data1 = json.load(json_file)
       
    for each_dictionary in all_data1:
        Space_TweetId = each_dictionary['id']
        Tweets = each_dictionary['text']
        Favourite_Tweets_count = each_dictionary['favorite_count']
        Follower_count = each_dictionary['user']['followers_count']
        Retweet_count = each_dictionary['retweet_count']
        TweetTime = each_dictionary['created_at']
        CompanyId = each_dictionary['user']['id']
        CompanyName =each_dictionary['user']['name']
        Description = each_dictionary['user']['description']
        Favorites_count = each_dictionary['user']['favourites_count']
        #Hashtags = each_dictionary['entities']['hashtags']       
        my_demo_list.append({'Space_TweetId': str(Space_TweetId),
                             'Tweets': str(Tweets),
                             'Favourite_Tweets_count': int(Favourite_Tweets_count),
                             'Retweet_count': int(Retweet_count),
                             'CompanyId': int(CompanyId),
                             #'Hashtags' : str(Hashtags),
                             'Follower_count' : int(Follower_count),
                             'CompanyName' : str(CompanyName),
                             'Description' : str(Description),
                             'TweetTime' :datetime.strptime(TweetTime,'%a %b %d %H:%M:%S +0000 %Y'),
                             'Favorites_count' : int(Favorites_count)
                            })
        

Company1 = pd.DataFrame(my_demo_list, columns = ['CompanyId', 'CompanyName','Description', 'Follower_count','Favorites_count','Space_TweetId', 'Tweets', 'TweetTime',
                                                   'Favourite_Tweets_count','Retweet_count' 
                                                   #'Hashtags'
                                                    
                                                    ]) 


In [303]:
#------------------------------
#Checking the first Normal Form
#-------------------------------
Company1 = pd.DataFrame(my_demo_list, columns = ['CompanyId', 'CompanyName','Description' ,'Follower_count','Favorites_count','Space_TweetId', 'Tweets', 'TweetTime',
                                                   'Favourite_Tweets_count','Retweet_count' 
                                                   #'Hashtags'
                                                    ]) 


#No Repeating groups = Done
#Data Values are atomic = Done
#Each field has different name = Done
#It has a primary key = Done

In [304]:
#----------------------------------------
#Checking the second normal form (2NF)
#-----------------------------------------
Company1 = pd.DataFrame(my_demo_list, columns = ['CompanyId', 'CompanyName','Description' ,'Follower_count','Favorites_count','Space_TweetId', 'Tweets', 'TweetTime',
                                                   'Favourite_Tweets_count','Retweet_count' 
                                                   #'Hashtags'
                                                    
                                                    ]) 
#Converting Company1 into second normal form

Space_profile = pd.DataFrame(my_demo_list, columns = ['CompanyId', 'CompanyName','Description','Follower_count','Favorites_count']) 

Space_Tweets  = pd.DataFrame(my_demo_list, columns = ['Space_TweetId', 'Tweets', 'TweetTime','Favourite_Tweets_count','Retweet_count'])

#It is in the First Normal Form =Done
#All non key attributes are dependent on ALL parts of primary key =Done
#Each Field has a unique name = done
#It has a primary key = done

In [309]:
#------------------------------
#Checking third normal form
#-------------------------------
Space_Tweets  = pd.DataFrame(my_demo_list, columns = ['Space_TweetId', 'Tweets', 'TweetTime','Favourite_Tweets_count','Retweet_count'])

#converting Tesla_Tweets to thrid normal form

S_tweets = pd.DataFrame(my_demo_list, columns = ['Space_TweetId', 'Tweets', 'TweetTime'])

S_tweets_info = pd.DataFrame(my_demo_list, columns = ['Space_TweetId', 'Favourite_Tweets_count','Retweet_count'])


#It should be in second normal form = Done
#All non-key attribute are not dependent on other non key attribute =Done
#Each field has a unique name= done
#Each field has a primary key = done

In [310]:
#-------------------------------------
# TABLE ON company SPACE X
#-------------------------------------
import sqlite3
conn = sqlite3.connect(':memory:')
c = conn.cursor()
Space_profile.to_sql('Space_profile', con=conn, if_exists='replace')
S_tweets.to_sql('S_tweets', con=conn, if_exists='replace')
S_tweets_info.to_sql('S_tweets_info', con=conn, if_exists='replace')


In [311]:
#-----------------------------
# VIEW ON company SPACE X
#-----------------------------


c.execute("CREATE VIEW IF NOT EXISTS Elon_Space AS SELECT ts.Tweets, ts.Space_TweetId,  ins.Retweet_count, ins.Favourite_Tweets_count FROM S_tweets_info ins JOIN S_tweets ts ON ins.Space_TweetId = ts.Space_TweetId")
pd.read_sql_query("SELECT  Space_TweetId, Retweet_count, Favourite_Tweets_count FROM Elon_Space limit 3", con=conn)

,Space_TweetId,Retweet_count,Favourite_Tweets_count
0,1104032250495004673,5592,39291
1,1104016731696054272,5942,0
2,1104015726812123137,6904,45177


In [312]:
#-----------------------------
# Index ON Space X
#-----------------------------

c.execute("CREATE INDEX IF NOT EXISTS IX_Elon_Space_Space_TweetId  ON S_tweets  (Space_TweetId ASC)")

In [313]:
#-----------------------------
# Function ON Space X
#-----------------------------


#c.execute("CREATE function Elon_space_funct (i_Space_TweetId number) RETURN varchar2 AS v_TweetTime varchar2(50); BEGIN SELECT TweetTime INTO v_TweetTime FROM S_tweets WHERE Space_TweetId = i_Space_TweetId; RETURN v_Location; END;")

In [314]:
#-----------------------------
# Procedure ON Space X
#-----------------------------


# c.execute("CREATE PROCEDURE IF NOT EXISTS sp_Retweet_count CURSOR c_Retweet_count IS SELECT ts.Tweets, ts.Space_TweetId,  ins.Retweet_count, ins.Favourite_Tweets_count FROM S_tweets_info ins JOIN S_tweets ts ON ins.Space_TweetId = ts.Space_TweetId begin FOR r_c_Retweet_count IN c_Retweet_count LOOP UPDATE Tweets SET Favourite_Tweets_count  = NULL ); END LOOP; END;" ) 


In [315]:
#-----------------------------------------------------------------------
# Retrieving Search data for User's company The Boring Company's timeline
# 
#-----------------------------------------------------------------------
my_list_of_dicts = []
for each_json_tweet in results3:
    my_list_of_dicts.append(each_json_tweet._json)
    with open('tweet_json.txt', 'w') as file:
        file.write(json.dumps(my_list_of_dicts, indent=4))
        my_demo_list= []
with open('tweet_json.txt', encoding='utf-8') as json_file:  
    all_data2 = json.load(json_file)
       
    for each_dictionary in all_data2:
        Bor_TweetId = each_dictionary['id']
        Tweets = each_dictionary['text']
        Favourite_Tweets_count = each_dictionary['favorite_count']
        Follower_count = each_dictionary['user']['followers_count']
        Retweet_count = each_dictionary['retweet_count']
        TweetTime = each_dictionary['created_at']
        CompanyId = each_dictionary['user']['id']
        CompanyName = each_dictionary['user']['name']
        Description = each_dictionary['user']['description']
        Favorites_count = each_dictionary['user']['favourites_count']
        
        #Hashtags = each_dictionary['entities']['hashtags']       
        my_demo_list.append({'Bor_TweetId': str(Bor_TweetId),
                             'Tweets': str(Tweets),
                             'Favourite_Tweets_count': int(Favourite_Tweets_count),
                             'Retweet_count': int(Retweet_count),
                             'CompanyId': int(CompanyId),
                             #'Hashtags' : str(Hashtags),
                             'Follower_count' : int(Follower_count),
                             'CompanyName' : str(CompanyName),
                             'Description' : str(Description),
                             'TweetTime' :datetime.strptime(TweetTime,'%a %b %d %H:%M:%S +0000 %Y'),
                             'Favorites_count' : int(Favorites_count),
                             'Follower_count': int(Follower_count)
                            })
        

Company2 = pd.DataFrame(my_demo_list, columns = ['CompanyId', 'CompanyName','Description','Follower_count', 'Bor_TweetId',
                                                 'Tweets','Favourite_Tweets_count','Retweet_count', 
                                                 'TweetTime','Favorites_count'
                                                   #'Hashtags'
                                                ]) 




In [316]:
#----------------------------------
#Checking the first Normal Form
#----------------------------------
Company2 = pd.DataFrame(my_demo_list, columns = ['CompanyId', 'CompanyName','Description','Follower_count', 'Bor_TweetId',
                                                 'Tweets','Favourite_Tweets_count','Retweet_count', 
                                                 'TweetTime','Favorites_count'
                                                   #'Hashtags'

                                                ]) 

#No Repeating groups = Done
#Data Values are atomic = Done
#Each field has different name = Done
#It has a primary key = Done


In [317]:
#--------------------------------------
#Checking the second normal form (2NF)
#---------------------------------------
Company2 = pd.DataFrame(my_demo_list, columns = ['CompanyId', 'CompanyName','Description','Follower_count', 'Bor_TweetId',
                                                 'Tweets','Favourite_Tweets_count','Retweet_count', 
                                                 'TweetTime','Favorites_count'
                                                ]) 

#Converting Company2 dataframe into second normal form

Boring_profile = pd.DataFrame(my_demo_list, columns = ['CompanyId', 'CompanyName','Description','Follower_count', 'Favorites_count']) 

Boring_info  = pd.DataFrame(my_demo_list, columns = [ 'Bor_TweetId','Tweets','Favourite_Tweets_count','Retweet_count', 'TweetTime']) 


#It is in the First Normal Form =Done
#All non key attributes are dependent on ALL parts of primary key =Done
#Each Field has a unique name = done
#It has a primary key = done


In [318]:
#------------------------------
#Checking third normal form
#-------------------------------

Boring_info = pd.DataFrame(my_demo_list, columns = [ 'Bor_TweetId','Tweets', 'TweetTime','Favourite_Tweets_count','Retweet_count'])

#converting Boring info into third normal form

B_tweets = pd.DataFrame(my_demo_list, columns = [ 'Bor_TweetId','Tweets', 'TweetTime'])

B_tweets_info = pd.DataFrame(my_demo_list, columns =['Bor_TweetId', 'Favourite_Tweets_count','Retweet_count' ])


#It should be in second normal form = Done
#All non-key attribute are not dependent on other non key attribute =Done
#Each field has a unique name= done
#Each field has a primary key = done


In [319]:
#-------------------------------------
# TABLE ON company THE BORING COMPANY
#-------------------------------------
import sqlite3
conn = sqlite3.connect(':memory:')
c = conn.cursor()
Boring_profile.to_sql('Pay_profile', con=conn, if_exists='replace')
B_tweets.to_sql('B_tweets', con=conn, if_exists='replace')
B_tweets_info.to_sql('B_tweets_info', con=conn, if_exists='replace')


In [320]:
#-----------------------------
# VIEW ON company THE BORING COMPANY
#-----------------------------


c.execute("CREATE VIEW IF NOT EXISTS Elon_Boring AS SELECT tb.Tweets, tb.TweetTime, inb.Retweet_count, inb.Favourite_Tweets_count FROM B_tweets_info inb JOIN B_tweets tb ON inb.Bor_TweetId = tb.Bor_TweetId  ")
pd.read_sql_query("SELECT Tweets, TweetTime, Favourite_Tweets_count FROM Elon_Boring limit 3", con=conn)

,Tweets,TweetTime,Favourite_Tweets_count
0,Thank you to @LVCVA @LVCVAImpact – hope we can...,2019-03-06 22:51:54,4007
1,RT @LVCVAImpact: Exciting news: LVCVA is recom...,2019-03-06 18:40:59,0
2,TBC is hiring. Come dig with us! https://t.co/...,2019-01-10 21:41:38,2676


In [321]:
#-----------------------------
# Index ON The Boring Company
#-----------------------------

c.execute("CREATE INDEX IF NOT EXISTS IX_Elon_Boring_TweetTime  ON B_tweets  (Tweets,  TweetTime)")

In [322]:
#-----------------------------
# Function ON The Boring Company
#-----------------------------


#c.execute("CREATE function Elon_boring_funct (i_Bor_TweetId number) RETURN varchar2 AS v_TweetTime varchar2(50); BEGIN SELECT TweetTime INTO v_TweetTime FROM S_tweets WHERE Bor_TweetId = i_Bor_TweetId; RETURN v_TweetTime; END;")

In [323]:
#-----------------------------
# Procedure ON The Boring Company
#-----------------------------


# c.execute("CREATE PROCEDURE IF NOT EXISTS sp_TWEET_max CURSOR c_tweet_max IS SELECT max(tb.Tweets), tb.TweetTime, inb.Retweet_count, inb.Favourite_Tweets_count FROM B_tweets_info inb JOIN B_tweets tb ON inb.Bor_TweetId = tb.Bor_TweetId begin FOR r_c_tweet_max IN c_tweet_max LOOP UPDATE Tweets SET Tweets = ' ' ); END LOOP; END;" ) 


In [324]:
#-----------------------------------------------------------------------
# Retrieving Search data for User's company Paypal's timeline
# 
#-----------------------------------------------------------------------
my_list_of_dicts = []
for each_json_tweet in results4:
    my_list_of_dicts.append(each_json_tweet._json)
    with open('tweet_json.txt', 'w') as file:
        file.write(json.dumps(my_list_of_dicts, indent=4))
        my_demo_list= []
with open('tweet_json.txt', encoding='utf-8') as json_file:  
    all_data3 = json.load(json_file)
       
    for each_dictionary in all_data3:
        Pay_TweetId = each_dictionary['id']
        Tweets = each_dictionary['text']
        Favourite_Tweets_count = each_dictionary['favorite_count']
        Follower_count = each_dictionary['user']['followers_count']
        Retweet_count = each_dictionary['retweet_count']
        TweetTime = each_dictionary['created_at']
        CompanyId = each_dictionary['user']['id']
        CompanyName =each_dictionary['user']['name']
        Description =each_dictionary['user']['description']
        Favorites_count = each_dictionary['user']['favourites_count']
        #Hashtags = each_dictionary['entities']['hashtags']       
        my_demo_list.append({'Pay_TweetId': str(Pay_TweetId),
                             'Tweets': str(Tweets),
                             'Favourite_Tweets_count': int(Favourite_Tweets_count),
                             'Retweet_count': int(Retweet_count),
                             'CompanyId': int(CompanyId),
                             #'Hashtags' : str(Hashtags),
                             'Follower_count' : int(Follower_count),
                             'Favorites_count' : int(Favorites_count),
                             'CompanyName' : str(CompanyName),
                              'Description' : str(Description),
                             'TweetTime': datetime.strptime(TweetTime,'%a %b %d %H:%M:%S +0000 %Y')
                            })
        

Company3 = pd.DataFrame(my_demo_list, columns = ['CompanyId', 'CompanyName', 'Description' ,'Follower_count','Pay_TweetId',
                                                 'Tweets', 'TweetTime', 'Favorites_count','Retweet_count', 'Favourite_Tweets_count'
                                                 ]) 


In [325]:
Company3 = pd.DataFrame(my_demo_list, columns = ['CompanyId', 'CompanyName', 'Description' ,'Follower_count','Favorites_count',
                                                 'Pay_TweetId', 'Tweets', 'TweetTime',
                                                'Retweet_count', 'Favourite_Tweets_count'
                                                 ]) 

#Checking the first Normal Form
#No Repeating groups = Done
#Data Values are atomic = Done
#Each field has different name = Done
#It has a primary key = Done

In [326]:
Company3 = pd.DataFrame(my_demo_list, columns = ['CompanyId', 'CompanyName', 'Description' ,'Follower_count','Favorites_count',
                                                 'Pay_TweetId', 'Tweets', 'TweetTime',
                                                'Retweet_count', 'Favourite_Tweets_count'
                                                 ]) 
#Converting Company3 into second normal form
Pay_profile = pd.DataFrame(my_demo_list, columns = ['CompanyId', 'CompanyName', 'Description' ,'Follower_count','Favorites_count' ])

Pay_info = pd.DataFrame(my_demo_list, columns = ['Pay_TweetId', 'Tweets', 'TweetTime','Retweet_count', 'Favourite_Tweets_count'
                                                 ]) 



#Checking the second normal form (2NF)
#It is in the First Normal Form =Done
#All non key attributes are dependent on ALL parts of primary key =Done
#Each Field has a unique name = done
#It has a primary key = done


In [327]:
Pay_info = pd.DataFrame(my_demo_list, columns = ['Pay_TweetId', 'Tweets', 'TweetTime','Retweet_count', 'Favourite_Tweets_count'
                                                ]) 
#Converting Pay_info into third normal form
P_tweets = pd.DataFrame(my_demo_list, columns = ['Pay_TweetId', 'Tweets', 'TweetTime'])

P_tweets_info = pd.DataFrame(my_demo_list, columns = ['Pay_TweetId', 'Retweet_count', 'Favourite_Tweets_count'])

#Checking third normal form
#It should be in second normal form = Done
#All non-key attribute are not dependent on other non key attribute =Done
#Each field has a unique name= done
#Each field has a primary key = done

In [328]:
#-----------------------------
# TABLE ON company PAYPAL
#-----------------------------
import sqlite3
conn = sqlite3.connect(':memory:')
c = conn.cursor()
Pay_profile.to_sql('Pay_profile', con=conn, if_exists='replace')
P_tweets.to_sql('P_tweets', con=conn, if_exists='replace')
P_tweets_info.to_sql('P_tweets_info', con=conn, if_exists='replace')



In [329]:
#-----------------------------
# VIEW ON company PAYPAL
#-----------------------------
c.execute("CREATE VIEW IF NOT EXISTS Elon_Paypal AS SELECT tp.Tweets, tp.TweetTime, inp.Retweet_count FROM P_tweets_info inp JOIN P_tweets tp ON inp.Pay_TweetId = tp.Pay_TweetId  ")
pd.read_sql_query("SELECT Tweets, Retweet_count FROM Elon_Paypal limit 3", con=conn)

,Tweets,Retweet_count
0,If you need to stop a repeating payment on a b...,4
1,Have questions about #PayPal? Receive real-tim...,0
2,Don’t settle until you set foot in the perfect...,15


In [330]:
#-----------------------------
# Index ON Paypal
#-----------------------------

c.execute("CREATE INDEX IF NOT EXISTS IX_Elon_Paypal_Retweet_count  ON P_tweets_info  (Retweet_count DESC)")

In [331]:
#-----------------------------
# Function ON Paypal
#-----------------------------


#c.execute("CREATE function Elon_paypal_funct (i_Retweet_count number) RETURN varchar2 AS v_TweetTime varchar2(50); BEGIN SELECT TweetTime INTO v_TweetTime FROM S_tweets WHERE Pay_TweetId = i_Pay_TweetId; RETURN v_TweetTime; END;")

In [332]:
#-----------------------------
# Procedure ON Paypal
#-----------------------------


# c.execute("CREATE PROCEDURE IF NOT EXISTS sp_TweetTime CURSOR c_TweetTime IS SELECT tp.Tweets, tp.TweetTime, inp.Retweet_count FROM P_tweets_info inp JOIN P_tweets tp ON inp.Pay_TweetId = tp.Pay_TweetId  begin FOR r_TweetTime IN c_TweetTime LOOP UPDATE Tweets SET Tweets = ' ' ); END LOOP; END;" ) 


In [333]:
#-----------------------------------------------------------------------
# Retrieving Search data for User's company Tesla's timeline
# 
#-----------------------------------------------------------------------
my_list_of_dicts = []
for each_json_tweet in results1:
    my_list_of_dicts.append(each_json_tweet._json)
    with open('tweet_json.txt', 'w') as file:
        file.write(json.dumps(my_list_of_dicts, indent=4))
        my_demo_list= []
with open('tweet_json.txt', encoding='utf-8') as json_file:  
    all_data4 = json.load(json_file)
    
    for each_dictionary in all_data4:
        Tesla_TweetId = each_dictionary['id']
        Tweets = each_dictionary['text']
        Favourite_Tweets_count = each_dictionary['favorite_count']
        Follower_count = each_dictionary['user']['followers_count']
        Retweet_count = each_dictionary['retweet_count']
        TweetTime = each_dictionary['created_at']
        CompanyId = each_dictionary['user']['id']
        CompanyName =each_dictionary['user']['name']
        Description =each_dictionary['user']['description']
        Favorites_count = each_dictionary['user']['favourites_count']
        #Hashtags = each_dictionary['entities']['hashtags']       
        my_demo_list.append({'Pay_TweetId': str(Tesla_TweetId),
                             'Tweets': str(Tweets),
                             'Favourite_Tweets_count': int(Favourite_Tweets_count),
                             'Retweet_count': int(Retweet_count),
                             'CompanyId': int(CompanyId),
                             #'Hashtags' : str(Hashtags),
                             'Follower_count' : int(Follower_count),
                             'Favorites_count' : int(Favorites_count),
                             'CompanyName' : str(CompanyName),
                              'Description' : str(Description),
                             'TweetTime': datetime.strptime(TweetTime,'%a %b %d %H:%M:%S +0000 %Y')
                            })

Company4 = pd.DataFrame(my_demo_list, columns = ['CompanyId', 'CompanyName', 'Description' ,'Follower_count','Favorites_count',
                                                 'Tesla_TweetId', 'Tweets', 'TweetTime',
                                                'Retweet_count', 'Favourite_Tweets_count'
                                                 ]) 



In [334]:
Company4 = pd.DataFrame(my_demo_list, columns = ['CompanyId', 'CompanyName', 'Description' ,'Follower_count','Favorites_count',
                                                 'Tesla_TweetId', 'Tweets', 'TweetTime',
                                                'Retweet_count', 'Favourite_Tweets_count'
                                                 ]) 

#Checking the first Normal Form
#No Repeating groups = Done
#Data Values are atomic = Done
#Each field has different name = Done
#It has a primary key = Done

In [335]:
Company4 = pd.DataFrame(my_demo_list, columns = ['CompanyId', 'CompanyName', 'Description' ,'Follower_count','Favorites_count',
                                                 'Tesla_TweetId', 'Tweets', 'TweetTime',
                                                'Retweet_count', 'Favourite_Tweets_count'
                                                 ]) 
#converting Company4 ino second normal form

Tesla_profile = pd.DataFrame(my_demo_list, columns = ['CompanyId', 'CompanyName', 'Description' ,'Follower_count','Favorites_count'])

Tesla_info = pd.DataFrame(my_demo_list, columns = ['Tesla_TweetId', 'Tweets', 'TweetTime','Retweet_count', 'Favourite_Tweets_count', 'CompanyName' ]) 


#Checking the second normal form (2NF)
#It is in the First Normal Form =Done
#All non key attributes are dependent on ALL parts of primary key =Done
#Each Field has a unique name = done
#It has a primary key = done

In [336]:
Tesla_info = pd.DataFrame(my_demo_list, columns = ['Pay_TweetId', 'Tweets', 'TweetTime','Retweet_count', 'Favourite_Tweets_count', 'CompanyName' ]) 

#converting Company3 ino second normal form

T_tweets = pd.DataFrame(my_demo_list, columns = ['Pay_TweetId', 'Tweets', 'TweetTime', 'CompanyName'])

T_tweets_info = pd.DataFrame(my_demo_list, columns = ['Pay_TweetId', 'Retweet_count', 'Favourite_Tweets_count', 'CompanyName' ])

#Checking third normal form
#It should be in second normal form = Done
#All non-key attribute are not dependent on other non key attribute = Done
#Each field has a unique name= done
#Each field has a primary key = done

In [337]:
#-----------------------------
# TABLE ON company TESLA
#-----------------------------
import sqlite3
conn = sqlite3.connect(':memory:')
c = conn.cursor()
Tesla_profile.to_sql('Tesla_profile', con=conn, if_exists='replace')
T_tweets.to_sql('T_tweets', con=conn, if_exists='replace')
T_tweets_info.to_sql('T_tweets_info', con=conn, if_exists='replace')

In [338]:
#-----------------------------
# VIEW ON company TESLA
#-----------------------------
c.execute("CREATE VIEW IF NOT EXISTS Elon_Tesla AS SELECT tw.Tweets, inf.Favourite_Tweets_count FROM T_tweets_info inf JOIN T_tweets tw ON inf.Pay_TweetId = tw.Pay_TweetId")

pd.read_sql_query("SELECT * FROM Elon_Tesla limit 3", con=conn)

,Tweets,Favourite_Tweets_count
0,@wonderofscience Whoa 😮,110
1,https://t.co/dN919zrsi9,1821
2,https://t.co/JNQ9fjJL75,2007


In [339]:
#-----------------------------
# Index ON Tesla
#-----------------------------

c.execute("CREATE INDEX IF NOT EXISTS IX_Elon_Tesla_Favourite_Tweets_count  ON T_tweets_info  (Favourite_Tweets_count)")

In [340]:
#-----------------------------
# Function ON Tesla
#-----------------------------


#c.execute("CREATE function Elon_tesla_funct (i_Retweet_count number) RETURN varchar2 AS v_TweetTime varchar2(50); BEGIN SELECT TweetTime INTO v_TweetTime FROM S_tweets WHERE tesla_TweetId = i_tesla_TweetId; RETURN v_TweetTime; END;")

In [341]:
#-----------------------------
# Procedure ON Tesla
#-----------------------------


# c.execute("CREATE PROCEDURE IF NOT EXISTS sp_Favourite_Tweets_count CURSOR c_Favourite_Tweets_count IS SELECT tw.Tweets, inf.Favourite_Tweets_count FROM T_tweets_info inf JOIN T_tweets tw ON inf.Pay_TweetId = tw.Pay_TweetId begin FOR r_Favourite_Tweets_count IN c_Favourite_Tweets_count LOOP UPDATE Tweets SET Tweets = ' ' ); END LOOP; END;" ) 
